## Code For Frozen Bert Weights for Regression
- This model trains and tests solely on patients who did not die

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from multimodal import MultimodalDataset, LOSNetWeighted, collation
from sklearn.preprocessing import StandardScaler
from transformers import AutoModel
import pandas as pd
import numpy as np
from torch.utils.tensorboard import SummaryWriter

In [2]:
!nvidia-smi

Sat Apr  6 13:51:18 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.116.04   Driver Version: 525.116.04   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    Off  | 00000000:00:05.0 Off |                  Off |
| 49%   75C    P2   298W / 300W |   9073MiB / 49140MiB |     99%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
base_path = 'data/regression/no-outliers/partitioned'

static_train = pd.read_csv(f'{base_path}/static_alive_train.csv')
static_val = pd.read_csv(f'{base_path}/static_alive_val.csv')
static_test = pd.read_csv(f'{base_path}/static_alive_test.csv')

In [4]:
notes = pd.read_csv('data/notes_with_interval.csv')
notes_train = notes[notes['id'].isin(static_train['id'])]
notes_val = notes[notes['id'].isin(static_val['id'])]
notes_test = notes[notes['id'].isin(static_test['id'])]

In [5]:
dynamic = pd.read_csv('data/dynamic_cleaned.csv')
dynamic_train = dynamic[dynamic['id'].isin(static_train['id'])].copy()
dynamic_val = dynamic[dynamic['id'].isin(static_val['id'])].copy()
dynamic_test = dynamic[dynamic['id'].isin(static_test['id'])].copy()

In [6]:
features = ['aniongap', 'bicarbonate', 'bun', 'calcium', 'chloride', 'creatinine', 'glucose', 'sodium', 'potassium']

scaler = StandardScaler()

dynamic_train.loc[:, features] = scaler.fit_transform(dynamic_train[features])
dynamic_val.loc[:, features] = scaler.transform(dynamic_val[features])
dynamic_test.loc[:, features] = scaler.transform(dynamic_test[features])  

dynamic_train.head()

,id,charttime,aniongap,bicarbonate,bun,calcium,chloride,creatinine,glucose,sodium,potassium
0,26115624,9/7/50 0:22,-0.708463,-0.374389,-0.996759,-0.690290,1.445426,-0.732283,-0.544027,0.747069,-0.907270
6,21525925,2/10/45 5:40,-0.239813,-0.181126,-1.108497,-0.580698,0.587007,-0.835441,-0.360288,0.194625,-0.762188
10,20866960,1/16/20 0:00,-0.708463,-0.567653,-0.326331,-1.457436,0.873147,-0.629125,0.581379,-0.173670,-0.181863
12,21580342,7/24/31 6:20,0.228836,0.785193,-0.065609,1.720741,-0.557553,2.001405,-0.429190,0.010477,-0.762188
14,23640433,8/11/73 13:47,-0.942788,1.558249,-0.549807,-0.251921,-0.557553,1.537194,-0.452157,0.562921,-0.617107


### Dynamic train cleaning

In [7]:
id_lengths_train = dynamic_train['id'].value_counts().to_dict()
dynamic_train = dynamic_train.sort_values(by=['id', 'charttime'])
dynamic_train = dynamic_train.apply(lambda x: list(x[features]), axis=1).groupby(dynamic_train['id']).agg(list)

dynamic_train

id
20008098    [[0.6974861412093623, -0.9541809330660428, -0....
20013244    [[-0.4741383268767333, 0.5919296400622551, -0....
20015730    [[-0.9427881141111716, 0.20540199678018062, -0...
20021110    [[0.22883635397492402, 0.5919296400622551, -0....
20024177    [[0.6974861412093623, 0.012138175139143398, -0...
                                  ...                        
29988601    [[0.9318110348265813, -0.37438946814293106, 0....
29990184    [[-0.005488539642295093, 1.9447763915495158, 0...
29990494    [[0.9318110348265813, -0.7609171114250055, -0....
29994296    [[-0.4741383268767333, -0.9541809330660428, 1....
29997500    [[-0.7084632204939524, 2.7178316781136647, -0....
Length: 1936, dtype: object

### Dynamic val cleaning

In [8]:
id_lengths_val = dynamic_val['id'].value_counts().to_dict()
dynamic_val = dynamic_val.sort_values(by=['id', 'charttime'])
dynamic_val = dynamic_val.apply(lambda x: list(x[features]), axis=1).groupby(dynamic_val['id']).agg(list)

dynamic_val

id
20031816    [[0.4631612475921431, 1.5582487482674412, -0.4...
20055925    [[-0.7084632204939524, 0.20540199678018062, -1...
20084622    [[0.22883635397492402, 0.9784572833443296, 0.3...
20099764    [[-0.005488539642295093, -0.7609171114250055, ...
20161146    [[0.9318110348265813, -1.7272362196301916, 2.5...
                                  ...                        
29457834    [[1.1661359284438004, -0.7609171114250055, 1.3...
29519314    [[-1.1771130077283907, -0.18112564650189383, 1...
29598220    [[0.6974861412093623, -1.3407085763481172, 0.3...
29744920    [[-1.1771130077283907, -0.37438946814293106, -...
29793270    [[-0.2398134332595142, -0.18112564650189383, -...
Length: 216, dtype: object

### Dynamic test cleaning

In [9]:
id_lengths_test = dynamic_test['id'].value_counts().to_dict()
dynamic_test = dynamic_test.sort_values(by=['id', 'charttime'])
dynamic_test = dynamic_test.apply(lambda x: list(x[features]), axis=1).groupby(dynamic_test['id']).agg(list)

dynamic_test

id
20020562    [[-0.9427881141111716, 0.012138175139143398, -...
20026358    [[1.6347857156782386, -1.7272362196301916, 0.4...
20048978    [[1.1661359284438004, -0.9541809330660428, 1.1...
20076522    [[-0.2398134332595142, 0.20540199678018062, -1...
20099311    [[-0.4741383268767333, 0.5919296400622551, -1....
                                  ...                        
29756347    [[1.1661359284438004, -0.9541809330660428, 0.5...
29800204    [[0.9318110348265813, 0.9784572833443296, 0.45...
29855398    [[-0.9427881141111716, -0.18112564650189383, -...
29921759    [[-0.9427881141111716, 0.7851934617032923, -0....
29961239    [[-0.9427881141111716, 1.1717211049853669, -0....
Length: 240, dtype: object

In [10]:
notes = notes[['id', 'charttime', 'text', 'interval']]

notes_train = notes[notes['id'].isin(static_train['id'])].copy()
notes_val = notes[notes['id'].isin(static_val['id'])].copy()
notes_test = notes[notes['id'].isin(static_test['id'])].copy()

In [11]:
train_data = MultimodalDataset(static=static_train, dynamic=dynamic_train, id_lengths=id_lengths_train, notes=notes_train)
validation_data = MultimodalDataset(static=static_val, dynamic=dynamic_val, id_lengths=id_lengths_val, notes=notes_val)

train_loader = DataLoader(train_data, batch_size=1500, shuffle=True, collate_fn=collation)
val_loader = DataLoader(validation_data, batch_size=150, shuffle=True, collate_fn=collation)

In [12]:
text_model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

for params in text_model.parameters():
    params.requires_grad = False

model = LOSNetWeighted(input_size=9, out_features=1, hidden_size=128, text_model=text_model)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

print(f'device: {device}')

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


device: cuda


In [13]:
class RMSELoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = nn.MSELoss()
    
    def forward(self, predicted, actual):
        return torch.sqrt(self.mse(predicted, actual))

In [14]:
criterion = RMSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [15]:
!nvidia-smi

Sat Apr  6 13:51:37 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.116.04   Driver Version: 525.116.04   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    Off  | 00000000:00:05.0 Off |                  Off |
| 49%   75C    P2   298W / 300W |  10711MiB / 49140MiB |    100%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [16]:
loss_base_path = 'losses/no-outliers/alive_regression'

In [17]:
epochs = 200
training_loss = []
validation_loss = []
patience = 10
stagnation = 0

for epoch in range(1, epochs+1):
    print(f'training epoch: [{epoch}/{epochs}]')
    model.train()
    training_loss_epoch = 0

    for step, batch in enumerate(train_loader):
        packed_dynamic_X, notes_X, notes_intervals, los = batch

        packed_dynamic_X = packed_dynamic_X.to(device)
        los = los.to(device)

        notes_X_gpu = []
        for notes in notes_X:
            notes_gpu = {key: value.to(device) for key, value in notes.items()}
            notes_X_gpu.append(notes_gpu)

        outputs = model(packed_dynamic_X, notes_X_gpu, notes_intervals)

        loss = criterion(outputs, los)
        training_loss_epoch += loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if step % max(1, round(len(train_loader) * 0.1)) == 0:
            print(f'step: [{step+1}/{len(train_loader)}] | loss: {loss.item():.4}')

            if step+1 == 1 and epoch == 1:
                with open(f'{loss_base_path}/loss_step.txt', 'w') as loss_step_f:
                    loss_step_f.write(f'{loss.item():.4f}\n')

            else:
                with open(f'{loss_base_path}/loss_step.txt', 'a') as loss_step_f:
                    loss_step_f.write(f'{loss.item():.4f}\n')

    avg_training_loss_epoch = training_loss_epoch / len(train_loader)
    training_loss.append(avg_training_loss_epoch.item())
    print(f'Training epoch loss: {avg_training_loss_epoch.item():.4f}\n')

    if epoch == 1:
        with open(f'{loss_base_path}/training_loss_epoch.txt', 'w') as loss_epoch_train_f:
            loss_epoch_train_f.write(f'{avg_training_loss_epoch.item():.4f}\n')

    else:
        with open(f'{loss_base_path}/training_loss_epoch.txt', 'a') as loss_epoch_train_f:
            loss_epoch_train_f.write(f'{avg_training_loss_epoch.item():.4f}\n')

    print(f'validation epoch: [{epoch}/{epochs}]')
    
    model.eval()
    with torch.no_grad():
        validation_loss_epoch = 0

        for val_step, val_batch in enumerate(val_loader):
            packed_dynamic_X_val, notes_X_val, notes_intervals_val, los_val = val_batch

            packed_dynamic_X_val = packed_dynamic_X_val.to(device)
            los_val = los_val.to(device)

            notes_X_val_gpu = []
            for notes in notes_X_val:
                notes_val_gpu = {key: value.to(device) for key, value in notes.items()}
                notes_X_val_gpu.append(notes_val_gpu)

            val_outputs = model(packed_dynamic_X_val, notes_X_val_gpu, notes_intervals_val)
            val_loss = criterion(val_outputs, los_val)
            validation_loss_epoch += val_loss

        avg_validation_loss = validation_loss_epoch / len(val_loader)
        print(f'Validation epoch loss: {avg_validation_loss.item():.4f}\n')
        
        if len(validation_loss) == 0 or (avg_validation_loss.item() < min(validation_loss)):
            stagnation = 0
            torch.save(model.state_dict(), 'saved-models/no-outliers/alive_regression_best_model.pth')
            print(f'new minimum validation loss')
            print(f'model saved\n')

        else:
            stagnation += 1

        validation_loss.append(avg_validation_loss.item())

        if epoch == 1:
            with open(f'{loss_base_path}/validation_loss_epoch.txt', 'w') as loss_epoch_val_f:
                loss_epoch_val_f.write(f'{avg_validation_loss.item():.4f}\n')

        else:
            with open(f'{loss_base_path}/validation_loss_epoch.txt', 'a') as loss_epoch_val_f:
                loss_epoch_val_f.write(f'{avg_validation_loss.item():.4f}\n')

        if stagnation >= patience:
            print(f'No improvement over {patience} epochs')
            print('Early stopping')
            break

    model.train()

    print('===============================\n')
    

training epoch: [1/200]
step: [1/2] | loss: 4.047
step: [2/2] | loss: 3.498
Training epoch loss: 3.7728

validation epoch: [1/200]
Validation epoch loss: 3.1234

new minimum validation loss
model saved


training epoch: [2/200]
step: [1/2] | loss: 3.133
step: [2/2] | loss: 2.619
Training epoch loss: 2.8759

validation epoch: [2/200]
Validation epoch loss: 2.6511

new minimum validation loss
model saved


training epoch: [3/200]
step: [1/2] | loss: 2.601
step: [2/2] | loss: 2.12
Training epoch loss: 2.3603

validation epoch: [3/200]
Validation epoch loss: 2.4599

new minimum validation loss
model saved


training epoch: [4/200]
step: [1/2] | loss: 2.485
step: [2/2] | loss: 2.434
Training epoch loss: 2.4595

validation epoch: [4/200]
Validation epoch loss: 2.5732


training epoch: [5/200]
step: [1/2] | loss: 2.629
step: [2/2] | loss: 2.662
Training epoch loss: 2.6459

validation epoch: [5/200]
Validation epoch loss: 2.6390


training epoch: [6/200]
step: [1/2] | loss: 2.689
step: [2/2] |